In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Project/Dacon/lgauto/open.zip
# !git clone --recursive https://github.com/Microsoft/LightGBM
# !cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
!pip install optuna
!pip install catboost
!pip install skranger
!pip install ngboost

Archive:  /content/drive/MyDrive/Project/Dacon/lgauto/open.zip
   creating: meta/
  inflating: meta/x_feature_info.csv  
  inflating: meta/y_feature_info.csv  
  inflating: meta/y_feature_spec_info.csv  
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 81 kB 10.4 MB/s 
     |████████████████████████████████| 209 kB 72.7 MB/s 
     |████████████████████████████████| 78 kB 7.6 MB/s 
     |████████████████████████████████| 112 kB 90.1 MB/s 
     |████████████████████████████████| 147 kB 84.2 MB/s 
     |████████████████████████████████| 49 kB 7.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=bed6673c988b0228c1bc0f6eecb542e6b4e4e16f35e0df2fead5790da8f3b63f
  Stored in directory: /roo

In [1]:
import pandas as pd
import random
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tqdm

from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold

from lightgbm import LGBMRegressor
from ngboost import NGBRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet, LinearRegression, Lasso, Ridge
from catboost import CatBoostRegressor, Pool
from skranger.ensemble import RangerForestRegressor
from sklearn.neighbors import RadiusNeighborsRegressor

from hyperopt import fmin, hp, tpe
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

In [2]:
class Config:
  seed = 42
  epochs = 200

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [4]:
def dataset_split_X_y(df):    
    """
    @Description: split data into features and labels
    @Param: df, pandas dataframe with columns starting with X for features and Y for labels
    @Return: features and labels in pandas dataframes
    """
    xs = df.filter(regex='X') # Input : X Feature
    ys = df.filter(regex='Y') # Output : Y Feature
    return xs, ys

In [5]:
def check_for_NAs(df, show=False):
    """
    @Description: checks for the NAs in the dataframe
    @Param1: df, pandas dataframe
    @Param2: show, boolean indicating whether NaN data are also necessary as a part of the output
    @Return: name of the columns with NaN
    """
    nan_values = df.loc[:, df.isnull().any()]
    if show:
        return df[df.isna().any(axis=1)]
    return list(nan_values.columns)

In [6]:
def check_for_label_bound(df, labels, bound):
    """
    @Description: check bound is inbetween min and max
    @Param1: df, pandas dataframe
    @Param2: labels, list of column names 
    @Param3: thres: list of bounds
    @Return: names of the columns not within the bound
    """
    n = len(labels)
    result = []
    for idx in range(n):
        col = labels[idx]
        thres = bound[idx]
        extracted_column = df[col]
        if not extracted_column.between(thres[0], thres[1]).all():
            result.append(labels[idx])
    if len(result) == 0:
        print('everything is within the bound')
    return result

In [7]:
def zero_variance(df):
    """
    @Description: check for zero_variance
    @Param1: df, pandas dataframe
    @Return: names of the columns with zero variance
    """
    result = []
    for col in df.columns:
        if df[col].var() == 0:
            result.append(col)
    return result

In [8]:
def get_top_correlation(df, n=10):
    """
    @Description: print out top correlated features
    @Param1: df, pandas dataframe
    @Param2: n, number of lines to print 
    @Return: pandas series
    """
    pairs = set()
    for idx1 in range(0, df.shape[1]):
        for idx2 in range(0, idx1+1):
            pairs.add((df.columns[idx1], df.columns[idx2]))
    corr = df.corr().abs().unstack()
    corr = corr.drop(labels=pairs).sort_values(ascending=False)
    return corr[0:n]

In [9]:
def adjacent_histogram_boxplot(feature_var, figsize = (7, 5)):
    """
    @Description: plot histogram and boxplot in next to each other
    @Param1: feature_var, pandas series 
    @Param2: figsize, size of the figure 
    """
    fig, (hist_plot, box_plot) = plt.subplots(nrows=2, sharex=True, gridspec_kw={'height_ratios':(.85,.15)}, figsize=figsize)
    sns.distplot(feature_var, kde=True, ax=hist_plot, kde_kws= {"linewidth":1.5}) 
    sns.boxplot(feature_var, ax=box_plot, linewidth = 1, width = 0.5)
    hist_plot.set_ylabel('')    
    hist_plot.set_xlabel('')
    box_plot.set_xlabel('')
    hist_plot.tick_params(labelsize=8)
    box_plot.tick_params(labelsize=8)
    fig.suptitle(feature_var.name, fontsize = 10)
    hist_plot.axvline(np.mean(feature_var),color='red',linestyle='-',lw = 1.5)
    hist_plot.axvline(np.median(feature_var),color='green',linestyle='--',lw = 1.5)
    

In [10]:
def lg_nrmse(gt, preds):
    """
    @Description: Metric used in this project
    @Params1: gt, pandas dataframe
    @Param2: preds, pandas dataframe
    @Return: nrmse score
    """
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    preds = pd.DataFrame(preds)
    all_nrmse = []
    for idx in range(0,14):
        rmse = mean_squared_error(gt.iloc[:,idx], preds.iloc[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [11]:
def lg_individual_nrmse(gt, preds):
    """
    @Description: Metric used in this project (individual)
    @Params1: gt, pandas dataframe
    @Param2: preds, pandas dataframe
    @Return: nrmse score
    """
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    rmse = mean_squared_error(gt, preds, squared=False)
    nrmse = rmse/np.mean(np.abs(gt))
    return nrmse

In [12]:
def find_outlier_zscore(data, threshold = 3):
    mean = np.mean(data)
    std = np.std(data)
    zs = [(y - mean) / std for y in data]
    masks = np.where(np.abs(zs) > threshold)
    return masks[0]

In [13]:
ys = ['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 
      'Y_06', 'Y_07', 'Y_08', 'Y_09', 'Y_10', 
      'Y_11', 'Y_12', 'Y_13', 'Y_14']
ys_bounds = [[0.2, 2], [0.2, 2.1], [0.2, 2.1], 
             [7, 19], [22, 36.5], [-19.2, 19], 
             [2.4, 4], [-29.2, -24], [-29.2, -24],
             [-30.6, -20], [19.6, 26.6], [-29.2, -24],
             [-29.2, -24], [-29.2, -24]]

In [14]:
seed_everything(Config.seed)

train_df = pd.read_csv('data/train.csv')
test_x = pd.read_csv('data/test.csv')
train_x, train_y = dataset_split_X_y(train_df)

cols_with_zero_variance = zero_variance(train_x) # 분산이 0 (통과 여부)
train_x = train_x.drop(cols_with_zero_variance, axis = 1)
test_x = test_x.drop(cols_with_zero_variance, axis = 1)

train_x = train_x.drop(['X_10', 'X_11'], axis = 1) # 결측치가 많음 (결측치 = 0, 공지사항)
test_x = test_x.drop(['X_10', 'X_11'], axis = 1)

test_x = test_x.drop('ID', axis=1)

In [15]:
df_indicator = pd.DataFrame()
for idx in range(len(ys)):
    y_series = ~train_y[ys[idx]].between(ys_bounds[idx][0], ys_bounds[idx][1])
    df_indicator = pd.concat([df_indicator, y_series.astype(int)], axis = 1)

In [16]:
lst = []
for i in df_indicator.columns:
    lst.append(df_indicator[df_indicator[i] == 1].index)
ans=set()
for i in lst:
    for k in i:
        ans.add(k)
print(len(ans))

ans = list(ans)
ans.sort()
spec_x = train_x.loc[ans, :]

3917


In [17]:
train_x_normal = train_x.drop(spec_x.index)
train_y_normal = train_y.drop(spec_x.index)

train_x_spec = train_x.drop(train_x_normal.index)
train_y_spec = train_y.drop(train_y_normal.index)

In [18]:
len(train_y_spec)

3917

In [19]:
class Config:
  seed = 42
  epochs = 200
  cv=10
  test_size = 0.2

In [22]:
def lgbm_objective(params):
    params = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'num_leaves': int(params['num_leaves']),
        'min_child_samples': int(params['min_child_samples']),
        'colsample_bytree': '{:.5f}'.format(params['colsample_bytree']),
        'subsample': '{:.5f}'.format(params['subsample']),
        'min_split_gain': '{:.5f}'.format(params['min_split_gain']),
        'scale_pos_weight': '{:.5f}'.format(params['scale_pos_weight']),
        'reg_alpha': '{:.5f}'.format(params['reg_alpha']),
        'reg_lambda': '{:.5f}'.format(params['reg_lambda']),
        'learning_rate': '{:.5f}'.format(params['learning_rate']),   
    }

    model = LGBMRegressor(
        n_jobs = -1,
        random_state = 1,
        verbose = 100,
        **params
    )

    losses = np.sqrt(-cross_val_score(model, train_x_normal, train_y['Y_01'], cv=Config.cv, scoring='neg_mean_squared_error'))
    losses = losses / np.mean(np.abs(train_y['Y_01']))
    
    print("NRMSE Loss {:.5f} params {}".format(losses.mean(), params))
    return losses.mean()


def xgb_objective(params):
    params = {

    }

    model = XGBRegressor(
        n_jobs = -1,
        verbose = 100,
        random_state = 1,
        **params
    )

    losses = np.sqrt(-cross_val_score(model, train_x, train_y['Y_01'], cv=Config.cv, scoring='neg_mean_squared_error'))
    losses = losses / np.mean(np.abs(train_y['Y_01']))
    return losses.mean()


def cat_objective(params):
    params = {
        'n_estimators': int(params['n_estimators']),
        'depth': int(params['depth']),
        'learning_rate': params['learning_rate'],   
        'l2_leaf_reg': params['l2_leaf_reg'],
        'max_bin': int(params['max_bin']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
        'random_strength': params['random_strength'],
        'fold_len_multiplier': params['fold_len_multiplier'],
        
    }

    model = CatBoostRegressor(
        logging_level='Silent',
        **params
    )

    losses = np.sqrt(-cross_val_score(model, train_x, train_y['Y_04'], cv=Config.cv, scoring='neg_mean_squared_error'))
    losses = losses / np.mean(np.abs(train_y['Y_04']))
    return losses.mean()

def extra_objective(params):
    params = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'min_samples_split': int(params['min_samples_split']),
        'min_samples_leaf': int(params['min_samples_leaf']),
        'min_weight_fraction_leaf': params['min_weight_fraction_leaf'],
        'max_features': params['max_features'],
        'max_leaf_nodes': int(params['max_leaf_nodes']),
        'min_impurity_decrease': params['min_impurity_decrease'],
        'bootstrap': params['bootstrap'],
        'ccp_alpha': params['ccp_alpha'],  
    }

    model = ExtraTreesRegressor(
        n_jobs = -1,
        verbose = 0,
        random_state = 1,
        **params
    )

    losses = np.sqrt(-cross_val_score(model, train_x, train_y['Y_01'], cv=Config.cv, scoring='neg_mean_squared_error'))
    losses = losses / np.mean(np.abs(train_y['Y_01']))
    return losses.mean()

def ngbr_objective(params):
    params = {
        'n_estimators': int(params['n_estimators']),
        'learning_rate': params['learning_rate'],
        'natural_gradient': params['natural_gradient'],
        'col_sample': float(params['col_sample']),
        'minibatch_frac': float(params['minibatch_frac']),
        'tol': float(params['tol']),
    }

    model = NGBRegressor(
        verbose = 100,
        random_state = 1,
        **params
    )

    losses = np.sqrt(-cross_val_score(model, train_x, train_y['Y_01'], cv=Config.cv, scoring='neg_mean_squared_error'))
    losses = losses / np.mean(np.abs(train_y['Y_01']))
    print("NRMSE Loss {:.5f} params {}".format(losses.mean(), params))
    return losses.mean()

## Catboost Regressor

In [ ]:
## https://catboost.ai/en/docs/concepts/parameter-tuning (참고)
space_catboost = {
    'n_estimators' : hp.quniform('n_estimators', 100, 300, 50),
    'depth': hp.quniform("depth", 2, 16, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 3, 8),
    'max_bin' : hp.quniform('max_bin', 1, 254, 1),
    'min_data_in_leaf' : hp.quniform('min_data_in_leaf', 2, 700, 1),
    'random_strength' : hp.loguniform('random_strength', np.log(0.005), np.log(5)),
    'fold_len_multiplier' : hp.loguniform('fold_len_multiplier', np.log(1.01), np.log(2.5)),
}

best = fmin(fn = cat_objective,
            space = space_catboost,
            algo = tpe.suggest,
            verbose = 1,
            max_evals = 200)


100%|██████████| 200/200 [35:40<00:00, 10.70s/it, best loss: 0.1916584045864259]


In [ ]:
print(best)

## Extra Trees Regressor

In [ ]:
space_extra = {
    'n_estimators' : hp.quniform('n_estimators', 100, 1500, 50),
    'max_depth': hp.quniform('max_depth', 3, 50, 1),
    'min_samples_split': hp.quniform('min_samples_split', 5, 50, 5),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 50, 1),
    'min_weight_fraction_leaf': hp.uniform('min_weight_fraction_leaf', 0.01, 0.5),
    'max_features': hp.choice('max_features', ['sqrt', 'log2', None, 'auto']),
    'max_leaf_nodes': hp.quniform('max_leaf_nodes', 3, 30, 1),
    'min_impurity_decrease': hp.uniform('min_impurity_decrease', 0, 200),
    'bootstrap':  hp.choice('bootstrap', [True, False]),
    'ccp_alpha': hp.uniform('ccp_alpha', 0.01, 1.0),
}

best = fmin(fn = extra_objective,
            space = space_extra,
            algo = tpe.suggest,
            verbose = 1,
            max_evals = 2)

best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['max_leaf_nodes'] = int(best['max_leaf_nodes'])


##LGBM

In [ ]:
space_lgbm = {
    'n_estimators' : hp.quniform('n_estimators', 100, 1500, 1),
    'max_depth': hp.quniform('max_depth', 5, 250, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 200, 5),
    'min_child_samples': hp.quniform('min_child_samples', 10, 150, 5),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'subsample': hp.uniform('subsample', 0.3, 1.0),
    'min_split_gain': hp.uniform('min_split_gain', 0, 0.7),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 1, 10),
    'reg_alpha': hp.uniform('reg_alpha', 0, 500),
    'reg_lambda': hp.uniform('reg_lambda', 0, 500),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
}

best = fmin(fn = lgbm_objective,
            space = space_lgbm,
            algo = tpe.suggest,
            verbose = 10,
            max_evals = 200)

print(best)
best['n_estimators'] = int(best['n_estimators'])
best['num_leaves'] = int(best['num_leaves'])
best['max_depth'] = int(best['max_depth'])
best['min_child_samples'] = int(best['min_child_samples'])

In [23]:
best = {'colsample_bytree': 0.572280100273023, 'learning_rate': 0.010283635038627429, 'max_depth': 180, 'min_child_samples': 135, 'min_split_gain': 0.04511227284338413, 'n_estimators': 900, 'num_leaves': 70, 'reg_alpha': 4.406681827912319, 'reg_lambda': 20.4785600448913, 'scale_pos_weight': 8.302374117433086, 'subsample': 0.1688669888026464}


val2 = lgbm_objective(best)

ValueError: Found input variables with inconsistent numbers of samples: [35690, 39607]

In [26]:
ys = ['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 
      'Y_06', 'Y_07', 'Y_08', 'Y_09', 'Y_10', 
      'Y_11', 'Y_12', 'Y_13', 'Y_14']
ys_bounds = [[0.2, 2], [0.2, 2.1], [0.2, 2.1], 
             [7, 19], [22, 36.5], [-19.2, 19], 
             [2.4, 4], [-29.2, -24], [-29.2, -24],
             [-30.6, -20], [19.6, 26.6], [-29.2, -24],
             [-29.2, -24], [-29.2, -24]]

In [27]:
val

0.22270174567825513

In [28]:
df_indicator = pd.DataFrame()
for idx in range(len(ys)):
    y_series = ~train_y[ys[idx]].between(ys_bounds[idx][0], ys_bounds[idx][1])
    df_indicator = pd.concat([df_indicator, y_series.astype(int)], axis = 1)

In [29]:
lst = []
for i in df_indicator.columns:
    lst.append(df_indicator[df_indicator[i] == 1].index)
ans=set()
for i in lst:
    for k in i:
        ans.add(k)
print(len(ans))

ans = list(ans)
ans.sort()
spec_x = train_x.loc[ans, :]

3917


In [30]:
lst_spec_x = pd.DataFrame([0 for i in range(len(train_x))])
lst_spec_x.loc[spec_x.index] = 1

In [78]:
train_x_normal = train_x.drop(spec_x.index)
train_y_normal = train_y.drop(spec_x.index)

train_x_spec = train_x.drop(train_x_normal.index)
train_y_spec = train_y.drop(train_y_normal.index)


In [118]:
train_x_end = pd.concat([train_x_spec.loc[:4000], train_x_normal])
train_y_end = pd.concat([train_y_spec.loc[:4000], train_y_normal])

In [117]:
len(train_x_spec.loc[:4000])

477

In [141]:
def func(best, y_val):

    model = LGBMRegressor(
            n_jobs = -1,
            random_state = 1,
            **best
        )


    model.fit(train_x_normal, train_y_normal[y_val])
    preds = model.predict(test_x)

    return pd.DataFrame(preds)
    

In [142]:
l = func(best, 'Y_01')


In [143]:
l

,0
0,1.376983
1,1.447808
2,1.379149
3,1.411775
4,1.306533
...,...
39603,1.273483
39604,1.297378
39605,1.249871
39606,1.241047


In [163]:
end_pd = pd.DataFrame()

In [164]:
best = {'colsample_bytree': 0.572280100273023, 'learning_rate': 0.010283635038627429, 'max_depth': 180, 'min_child_samples': 135, 'min_split_gain': 0.04511227284338413, 'n_estimators': 900, 'num_leaves': 70, 'reg_alpha': 4.406681827912319, 'reg_lambda': 20.4785600448913, 'scale_pos_weight': 8.302374117433086, 'subsample': 0.1688669888026464}
end_pd = pd.concat([end_pd, func(best, 'Y_01')], axis=1)

In [165]:
best =  {'colsample_bytree': 0.7641322280477741, 'learning_rate': 0.010977205425053654, 'max_depth': 90, 'min_child_samples': 75, 'min_split_gain': 0.13379952895779884, 'n_estimators': 900, 'num_leaves': 80, 'reg_alpha': 1.9214119194170154, 'reg_lambda': 14.454450236504218, 'scale_pos_weight': 2.171961031806387, 'subsample': 0.9552593593877317}

end_pd = pd.concat([end_pd, func(best, 'Y_02')], axis=1)

In [166]:
best =  {'colsample_bytree': 0.5504769098255781,  'learning_rate': 0.019653385015120244, 'max_depth': 220, 'min_child_samples': 25, 'min_split_gain': 0.1273611040963466, 'n_estimators': 470, 'num_leaves': 160, 'reg_alpha': 3.5549669150756706, 'reg_lambda': 39.88636182674132, 'scale_pos_weight': 12.46696320152359, 'subsample': 0.7590007450921917}

end_pd = pd.concat([end_pd, func(best, 'Y_03')], axis=1)

In [167]:
best =  {'colsample_bytree': 0.5597537952569402, 'learning_rate': 0.02374663979814546, 'max_depth': 32, 'min_child_samples': 100, 'min_split_gain': 0.12211426885216736, 'n_estimators': 1263, 'num_leaves': 200, 'reg_alpha': 14.606693962963451, 'reg_lambda': 299.52278825209424, 'scale_pos_weight': 7.7785016838070735, 'subsample': 0.6254745287838821}

end_pd = pd.concat([end_pd, func(best, 'Y_04')], axis=1)

In [168]:
best =  {'colsample_bytree': 0.4311015575880258, 'learning_rate': 0.01749725932551278, 'max_depth': 53, 'min_child_samples': 15, 'min_split_gain': 0.2820951740673634, 'n_estimators': 974, 'num_leaves': 165, 'reg_alpha': 9.604623064885754, 'reg_lambda': 12.314490508636432, 'scale_pos_weight': 6.6422956907936825, 'subsample': 0.7390190399971659}

end_pd = pd.concat([end_pd, func(best, 'Y_05')], axis=1)

In [169]:
best =  {'colsample_bytree': 0.6889745043181079, 'learning_rate': 0.06146161938790444, 'max_depth': 89, 'min_child_samples': 10, 'min_split_gain': 0.669592868575692, 'n_estimators': 1169, 'num_leaves': 175, 'reg_alpha': 11.405277636150856, 'reg_lambda': 112.37954230084294, 'scale_pos_weight': 5.932435783263877, 'subsample': 0.8265223228903998}  

end_pd = pd.concat([end_pd, func(best, 'Y_06')], axis=1)

In [170]:
best =  {'colsample_bytree': 0.8663251864650988, 'learning_rate': 0.018110306887688978, 'max_depth': 166, 'min_child_samples': 50, 'min_split_gain': 0.025403061552667243, 'n_estimators': 1080, 'num_leaves': 100, 'reg_alpha': 2.0131018839563666, 'reg_lambda': 63.56640846106552, 'scale_pos_weight': 1.8584564419776715, 'subsample': 0.7643028435523616}
end_pd = pd.concat([end_pd, func(best, 'Y_07')], axis=1)

In [171]:
best =  {'colsample_bytree': 0.8970390757241629, 'learning_rate': 0.03571726260659087, 'max_depth': 164, 'min_child_samples': 30, 'min_split_gain': 0.2863362850926679, 'n_estimators': 740, 'num_leaves': 100, 'reg_alpha': 1.1167159754886287, 'reg_lambda': 280.9798636389436, 'scale_pos_weight': 4.75867892931176, 'subsample': 0.681716202670263}
end_pd = pd.concat([end_pd, func(best, 'Y_08')], axis=1)

In [172]:
best =  {'n_estimators': 900, 'max_depth': 86, 'num_leaves': 150, 'min_child_samples': 85, 'colsample_bytree': '0.90507', 'subsample': '0.62362', 'min_split_gain': '0.21034', 'scale_pos_weight': '8.77311', 'reg_alpha': '0.07069', 'reg_lambda': '499.10672', 'learning_rate': '0.04679'}
end_pd = pd.concat([end_pd, func(best, 'Y_09')], axis=1)

In [173]:
best =  {'colsample_bytree': 0.8350973419202665, 'learning_rate': 0.03134966396365972, 'max_depth': 114, 'min_child_samples': 20, 'min_split_gain': 0.24406788869557822, 'n_estimators': 454, 'num_leaves': 115, 'reg_alpha': 1.0870546166564243, 'reg_lambda': 346.21163772786895, 'scale_pos_weight': 5.81617865285278, 'subsample': 0.45612075761336973}
end_pd = pd.concat([end_pd, func(best, 'Y_10')], axis=1)

In [174]:
best =  {'colsample_bytree': 0.7285829045071064, 'learning_rate': 0.019839273085108612, 'max_depth': 71, 'min_child_samples': 50, 'min_split_gain': 0.35567737788276876, 'n_estimators': 970, 'num_leaves': 140, 'reg_alpha': 0.27353134227182774, 'reg_lambda': 157.85749037224548, 'scale_pos_weight': 5.956126991298146, 'subsample': 0.7509931500532172}
end_pd = pd.concat([end_pd, func(best, 'Y_11')], axis=1)

In [175]:
best =  {'colsample_bytree': 0.6115826698158419, 'learning_rate': 0.010052927231718068, 'max_depth': 71, 'min_child_samples': 85, 'min_split_gain': 0.12003011548878659, 'n_estimators': 1300, 'num_leaves': 120, 'reg_alpha': 1.3013867029804251, 'reg_lambda': 269.3915696845848, 'scale_pos_weight': 5.290961082236748, 'subsample': 0.7542724715058367}
end_pd = pd.concat([end_pd, func(best, 'Y_12')], axis=1)

In [176]:
best =  {'colsample_bytree': 0.9511047907962863, 'learning_rate': 0.023257873709858216, 'max_depth': 58, 'min_child_samples': 80, 'min_split_gain': 0.21488153574891886, 'n_estimators': 1300, 'num_leaves': 150, 'reg_alpha': 0.33761852089148814, 'reg_lambda': 57.05291849099506, 'scale_pos_weight': 2.0801436555772854, 'subsample': 0.5580106548214563}
end_pd = pd.concat([end_pd, func(best, 'Y_13')], axis=1)

In [177]:
best =  {'colsample_bytree': 0.8851122740930837, 'learning_rate': 0.013136814152245062, 'max_depth': 249, 'min_child_samples': 65, 'min_split_gain': 0.2072264172906347, 'n_estimators': 450, 'num_leaves': 135, 'reg_alpha': 0.642890771203696, 'reg_lambda': 45.624663648443345, 'scale_pos_weight': 6.400746088779947, 'subsample': 0.30084274480143686}
end_pd = pd.concat([end_pd, func(best, 'Y_14')], axis=1)

In [178]:
end_pd

,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.391697,1.154704,1.033526,14.465251,32.407712,16.742986,3.012424,-26.034182,-26.024085,-22.127878,24.553438,-25.990061,-25.936504,-25.961329
1,1.455883,1.181129,1.103510,13.448666,31.392519,16.695410,3.129448,-26.070582,-26.139123,-22.327433,24.402505,-26.038111,-26.120345,-26.043134
2,1.375828,1.096432,1.011833,14.629861,31.759342,16.852628,3.095265,-25.978228,-25.872189,-22.060548,24.737351,-25.836653,-25.801612,-25.885519
3,1.417349,1.148919,1.001002,15.230288,32.158597,17.151581,3.059559,-25.593811,-25.608469,-21.691326,25.014891,-25.569533,-25.607190,-25.615177
4,1.307132,1.012789,0.936047,15.011538,31.435335,16.969695,3.020458,-25.688307,-25.677593,-21.883398,24.846907,-25.595199,-25.622568,-25.694433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,1.284618,1.011609,1.020121,12.481210,30.706559,16.776067,3.186461,-26.345256,-26.365513,-22.692625,24.471625,-26.314099,-26.272776,-26.287771
39604,1.278071,0.909527,0.981409,14.619227,31.650746,16.756380,3.149365,-26.304200,-26.287231,-22.669750,24.486648,-26.230034,-26.248674,-26.273253
39605,1.283944,0.944038,0.964451,13.256931,31.108957,16.735006,3.136356,-26.438000,-26.375102,-22.712396,24.377857,-26.406290,-26.480849,-26.499461
39606,1.231212,0.885186,0.949269,13.759728,31.372014,16.849927,3.157621,-26.385419,-26.321913,-22.713094,24.681208,-26.260347,-26.268088,-26.305381


In [179]:
end_pd.to_csv('data/end_submit.csv')

##NGBR

In [ ]:
space_ngboost = {
    'n_estimators': hp.quniform('n_estimators', 100, 500, 10),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'natural_gradient': hp.choice('natural_gradient', [True, False]),
    'col_sample': hp.quniform('col_sample', 0, 1, 0.01),
    'minibatch_frac': hp.quniform('minibatch_frac', 0, 1, 0.01),
    'tol': hp.uniform('tol', 1e-6, 3e-4),
}

best = fmin(fn = ngbr_objective,
            space = space_ngboost,
            algo = tpe.suggest,
            verbose = 10,
            max_evals = 100)

print(best)
best['n_estimators'] = int(best['n_estimators'])

In [ ]:
def get_stacking_base_datasets(model, train_x, train_y, col,test, params):
    kf = KFold(n_splits=Config.cv, shuffle=False)
    train_fold_pred = np.zeros((train_x.shape[0],1))
    test_pred = np.zeros((test.shape[0],Config.cv))
    
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(train_x)):
        print('Fold : ', folder_counter, ' Start')
        X_tr = train_x.loc[train_index]
        y_tr = train_y[col].loc[train_index]
        X_te = train_x.loc[valid_index] 
        
        if model == 'cat':
          model = CatBoostRegressor(random_state=1,
                                    **params)
        
        elif model == 'extra':
          model = ExtraTreesRegressor(random_state=1, 
                                      **params)

        elif model == 'ngbr':
          model = NGBRegressor(random_state = 1)
        
        elif model == 'lgbm':
          model = LGBMRegressor(random_state=1, n_jobs=-1, 
                                **params)

        model.fit(X_tr, y_tr)
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1) 
        test_pred[:, folder_counter] = model.predict(test) 
        
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    return train_fold_pred, test_pred_mean 

In [ ]:
model = LGBMRegressor(
        n_jobs = -1,
        random_state = 1,
        verbose = 100,
        **best
    )


# model 8개
xx_train, xx_test = get_stacking_base_datasets(model, train_x, train_y, col='Y_01', test=test_x)
yy_train, yy_test = get_stacking_base_datasets(model, train_x, train_y, col='Y_01', test=test_x)
zz_train, zz_test = get_stacking_base_datasets(model, train_x, train_y, col='Y_01', test=test_x)
qq_train, qq_test = get_stacking_base_datasets(model, train_x, train_y, col='Y_01', test=test_x)

Stack_final_X_train = np.concatenate((xx_train,yy_train,zz_train,qq_train), axis=1)
Stack_final_X_test = np.concatenate((xx_test,yy_test,zz_test,qq_test), axis=1)

# final_model 선택해야함
final_model = None
final_model.fit(Stack_final_X_train, y_train)
stack_final = final_model.predict(Stack_final_X_test) 




In [ ]:
## col1 col2 지정
stack_final.to_csv(f'{col1}_{col2}.csv', index=False)